[View in Colaboratory](https://colab.research.google.com/github/MarcinWylot/CreditCardFraudDetectionKaggleDS/blob/master/CreditCardFraudDetectionKaggleDS.ipynb)

In [0]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
#from google.colab import files
#uploaded = files.upload()

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
#uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
#uploaded.SetContentString('Sample upload file content')
#uploaded.Upload()
#print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': '1QQara53mzPKgRO4poTIWtQ2Ijm3GcqkV'})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile('./creditcard.csv')
!ls


In [7]:
data = pd.read_csv('./creditcard.csv')
data = data.drop(['Time'], axis=1)
#data.describe()
X_train_full, X_test = train_test_split(data, test_size=0.2, random_state=0)

#X_train = data[data.Class == 0]
X_train = X_train_full
y_train = X_train['Class']
X_train = X_train.drop(['Class'], axis=1)
#X_train = X_train.values

y_test = X_test['Class']
X_test = X_test.drop(['Class'], axis=1)
#X_test = X_test.values


fraud = X_train_full[X_train_full['Class'] == 1]
valid = X_train_full[X_train_full['Class'] == 0]

fraud_fraction = len(fraud) / float(len(valid))
print(fraud_fraction)

0.0017190289025473282


In [3]:
#data.hist(figsize=(20,20))
#plt.show()


In [0]:
#fig = plt.figure(figsize = (12,8))
#sns.heatmap(data.corr(), vmax=.8, square=True)
#plt.show()

In [0]:
#from sklearn.preprocessing import StandardScaler

#data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

#columns = [c for c in X_train.columns.tolist() if c not in ["Class"]]

#X = X_train[columns]
#Y = X_train["Class"]

#print(X.shape, Y.shape)

In [67]:
from sklearn.ensemble import IsolationForest
clf_IsolationForest = IsolationForest(max_samples = len(X_test), contamination = fraud_fraction, random_state=1)

clf_IsolationForest.fit(X_train)
y_IsolationForest = clf_IsolationForest.predict(X_test)

y_IsolationForest[y_IsolationForest == 1] = 0
y_IsolationForest[y_IsolationForest == -1] = 1

print(accuracy_score(y_test,y_IsolationForest))
print(classification_report(y_test,y_IsolationForest))

0.9976475545100242
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     56861
          1       0.33      0.33      0.33       101

avg / total       1.00      1.00      1.00     56962



In [8]:
#it's not the best method to test unseen data 
from sklearn.neighbors import LocalOutlierFactor
clf_LocalOutlierFactor = LocalOutlierFactor(n_neighbors=20, contamination = fraud_fraction)

clf_LocalOutlierFactor.fit(X_train)
y_LocalOutlierFactor = clf_LocalOutlierFactor._decision_function(X_test)

print(y_LocalOutlierFactor)
#y_LocalOutlierFactor[y_LocalOutlierFactor == 1] = 0
#y_LocalOutlierFactor[y_LocalOutlierFactor == -1] = 1

#print(accuracy_score(y_test,y_LocalOutlierFactor))
#print(classification_report(y_test,y_LocalOutlierFactor))

[-1.09656393 -1.41517926 -1.35353435 ... -1.08916324 -1.74549618
 -1.01231912]


In [0]:
y_LocalOutlierFactor = clf_LocalOutlierFactor._decision_function(X_train)

In [63]:
threshold = -2.3

print(np.histogram(y_LocalOutlierFactor,bins=10))
#plt.plot(y_LocalOutlierFactor)
#plt.show()

y_pred = [1 if e < threshold else 0 for e in y_LocalOutlierFactor]
print(y_LocalOutlierFactor)
#print(y_test.tolist())
#plt.plot(y_pred)
#plt.plot(y_test)
plt.show()
#plt.hist(y_LocalOutlierFactor, bins=10) 
#plt.show()
print(classification_report(y_test.tolist(),y_pred))

(array([    1,     3,     8,     0,     5,    15,    14,     8,     9,
       56899]), array([-1.68557162e+10, -1.51701446e+10, -1.34845730e+10, -1.17990014e+10,
       -1.01134297e+10, -8.42785812e+09, -6.74228650e+09, -5.05671487e+09,
       -3.37114325e+09, -1.68557163e+09, -8.87050612e-01]))
[-1.09656393 -1.41517926 -1.35353435 ... -1.08916324 -1.74549618
 -1.01231912]
             precision    recall  f1-score   support

          0       1.00      0.99      1.00     56861
          1       0.05      0.16      0.07       101

avg / total       1.00      0.99      0.99     56962



In [57]:
#https://medium.com/@curiousily/credit-card-fraud-detection-using-autoencoders-in-keras-tensorflow-for-hackers-part-vii-20e0c85301bd
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers



data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

X_train = X_train_full[X_train_full.Class == 0] #we tain the model on normal transaction only 
y_train = X_train['Class']
X_train = X_train.drop(['Class'], axis=1)
X_train = X_train.values

input_dim = X_train.shape[1]
encoding_dim = 14

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)


nb_epoch = 20
batch_size = 32
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history

Train on 284315 samples, validate on 56962 samples
Epoch 1/100
284315/284315 [==============================] - 15s 54us/step - loss: 0.7991 - acc: 0.5911 - val_loss: 0.7769 - val_acc: 0.6398
Epoch 2/100
216352/284315 [=====================>........] - ETA: 3s - loss: 0.7465 - acc: 0.6487

284315/284315 [==============================] - 15s 52us/step - loss: 0.7391 - acc: 0.6502 - val_loss: 0.7642 - val_acc: 0.6524
Epoch 3/100
284315/284315 [==============================] - 15s 52us/step - loss: 0.7305 - acc: 0.6617 - val_loss: 0.7580 - val_acc: 0.6672
Epoch 4/100
 95872/284315 [=========>....................] - ETA: 8s - loss: 0.7209 - acc: 0.6646

284315/284315 [==============================] - 15s 52us/step - loss: 0.7255 - acc: 0.6674 - val_loss: 0.7551 - val_acc: 0.6621
Epoch 5/100
284315/284315 [==============================] - 15s 52us/step - loss: 0.7217 - acc: 0.6782 - val_loss: 0.7521 - val_acc: 0.6826
Epoch 6/100
 71200/284315 [======>.......................] - ETA: 10s - loss: 0.7075 - acc: 0.6835

284315/284315 [==============================] - 15s 52us/step - loss: 0.7192 - acc: 0.6898 - val_loss: 0.7487 - val_acc: 0.6905
Epoch 7/100
284315/284315 [==============================] - 15s 52us/step - loss: 0.7173 - acc: 0.6990 - val_loss: 0.7490 - val_acc: 0.6909
Epoch 8/100
 63520/284315 [=====>........................] - ETA: 10s - loss: 0.7315 - acc: 0.7014

284315/284315 [==============================] - 15s 53us/step - loss: 0.7157 - acc: 0.7020 - val_loss: 0.7455 - val_acc: 0.6972
Epoch 9/100
284315/284315 [==============================] - 15s 52us/step - loss: 0.7147 - acc: 0.7034 - val_loss: 0.7459 - val_acc: 0.6941
Epoch 10/100
 59936/284315 [=====>........................] - ETA: 10s - loss: 0.7178 - acc: 0.7028

...

284315/284315 [==============================] - 15s 52us/step - loss: 0.7062 - acc: 0.7141 - val_loss: 0.7393 - val_acc: 0.7086
Epoch 93/100
284315/284315 [==============================] - 15s 52us/step - loss: 0.7064 - acc: 0.7146 - val_loss: 0.7328 - val_acc: 0.7112
Epoch 94/100
 59136/284315 [=====>........................] - ETA: 10s - loss: 0.7065 - acc: 0.7183

284315/284315 [==============================] - 15s 52us/step - loss: 0.7063 - acc: 0.7145 - val_loss: 0.7339 - val_acc: 0.7135
Epoch 95/100
284315/284315 [==============================] - 15s 52us/step - loss: 0.7062 - acc: 0.7144 - val_loss: 0.7334 - val_acc: 0.7148
Epoch 96/100
 59712/284315 [=====>........................] - ETA: 10s - loss: 0.7079 - acc: 0.7139

284315/284315 [==============================] - 15s 53us/step - loss: 0.7060 - acc: 0.7142 - val_loss: 0.7352 - val_acc: 0.7042
Epoch 97/100
284315/284315 [==============================] - 15s 52us/step - loss: 0.7062 - acc: 0.7144 - val_loss: 0.7334 - val_acc: 0.7127
Epoch 98/100
 57856/284315 [=====>........................] - ETA: 10s - loss: 0.7186 - acc: 0.7185

284315/284315 [==============================] - 15s 52us/step - loss: 0.7065 - acc: 0.7140 - val_loss: 0.7355 - val_acc: 0.7081
Epoch 99/100
284315/284315 [==============================] - 15s 52us/step - loss: 0.7058 - acc: 0.7148 - val_loss: 0.7342 - val_acc: 0.7076
Epoch 100/100
 58368/284315 [=====>........................] - ETA: 10s - loss: 0.7119 - acc: 0.7143

284315/284315 [==============================] - 15s 52us/step - loss: 0.7062 - acc: 0.7143 - val_loss: 0.7343 - val_acc: 0.7070


In [59]:
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)

X_train = data
#X_train = X_train[X_train.Class == 0] #we tain the model on normal transaction only 
y_train = X_train['Class']
X_train = X_train.drop(['Class'], axis=1)
X_train = X_train.values

predictions = autoencoder.predict(X_train)

mse = np.mean(np.power(X_train - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_train})

threshold = 3
y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]

print(classification_report(error_df.true_class,y_pred))



             precision    recall  f1-score   support

          0       1.00      0.98      0.99    284315
          1       0.06      0.82      0.12       492

avg / total       1.00      0.98      0.99    284807

